This notebook is created at GESIS for general public to test <a href="https://github.com/serengil/deepface">Deepface</a> on gender inference from images. It is a supplement to the [GESIS Gender Inference website](http://193.175.238.89/Gender_Inference/images.html). You can upload your own image or use a url. All data is deleted afterwards.

1. Run all cells by clicking "Cell" -> "Run all" <br/>
2. Upload several photos from your computer by clicking "Upload" <br/>
3. Press "Submit" to run the model on your data. The outcome will be displayed in the table with predicted gender and confidence. The results are also saved to the csv file <br/>
4. Button "Clear" ensures no data is stored afterwards

<b>Important</b>: After clicking "Cell" -> "Run all", If the buttons are not appearing please restart the kernal or refresh the page.

In [1]:
import os
import sys
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Keras outputs warnings using `print` to stderr so let's direct that to devnull temporarily
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')


# Importing necessary libraries
from ipywidgets import VBox, HBox
import ipywidgets as widgets
from IPython.display import clear_output
from PIL import Image
from IPython.display import HTML
from pathlib import Path
from io import BytesIO
import json
import os
import requests as requests
from IPython.display import HTML
from deepface import DeepFace
from deepface.extendedmodels import  Gender
import shutil 
import csv
import pandas as pd
import pathlib
import shutil
from tqdm import tnrange

# Load the gender model weights
home = str(Path.home())
source = '../DeepFace/gender_model_weights.h5'
destination = home+'/.deepface/weights/gender_model_weights.h5'
if not pathlib.Path(destination).exists():
    if not pathlib.Path(home+'/.deepface/weights/').exists():
        os.mkdir(home + "/.deepface/")
        os.mkdir(home + "/.deepface/weights/")
    dest = shutil.copyfile(source, destination)

# Initiate a gender model
models = {}
models["gender"] = Gender.loadModel()
clear_output()

In [2]:
HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [3]:
# running deepface
def deepface(filename):
    deepface_pred = DeepFace.analyze(f"images/{filename}", actions=['gender'], enforce_detection=True)
    if deepface_pred['gender'] == 'Woman':
            gender = 'Female'
    if deepface_pred['gender'] == 'Man':
            gender = 'Male'
    return gender

Please upload images from your computer all at once.

In [4]:
# Set up layout for upload widgets


def initialize_widgets():
    global upload, layout
    upload = widgets.FileUpload(
        accept='image/*',
        multiple=True
    )
    layout = VBox([upload, submit])
    display(layout)

global clear, submit, submitted
submitted = False
submit = widgets.Button(description="Submit")
clear = widgets.Button(description="Clear")
initialize_widgets()

# Clear the output and delete the files


def on_clear_clicked(b):
    global submitted
    clear_output()
    initialize_widgets()
    for filename in os.listdir('./images/'):
        if filename.endswith(".csv"):
            os.remove(filename)
    try:
        shutil.rmtree('./images/')
    except:
        pass
    submitted = False

# Resize and convert images for M3. Save the file (is deleted afterwards)


def preprocess_and_save(content, filename):
    stream = BytesIO(content)
    image = Image.open(stream).convert("RGB")
    stream.close()
    image.save(f"images/{filename}")

# Submitting an image either as fileupload or from url


def on_submit_clicked(b):
    global submitted, content
    if not upload.value:   # no input is given
        print('No image has been uploaded')
        return False
    print('Please be patient, don\'t restart the kernel or refresh the page. The model may take some time to predict the gender.\n \n')
    file_list = list(upload.value.keys())
    if len(file_list) > 200:
        file_list = file_list[:200]
        print('The model will return the predictions only for the first 200 images.\n')
    Path("images/").mkdir(parents=True, exist_ok=True)
    with open('output.csv', 'w', newline='') as output:   # output file is created
        wr = csv.writer(output, quoting=csv.QUOTE_ALL)
        wr.writerow(['Filename', 'Gender'])
        for index in tnrange(len(file_list), desc ="Progress"):
            content = upload.value[file_list[index]]['content']   # extracting the bytes for images
            preprocess_and_save(content, file_list[index])
            try:
                pred_gender = deepface(file_list[index])
                wr = csv.writer(output, quoting=csv.QUOTE_ALL)
                wr.writerow([file_list[index], pred_gender])
            except:
                wr = csv.writer(output, quoting=csv.QUOTE_ALL)
                wr.writerow([file_list[index], 'NaN'])    # if there is an error in returning gender , write undefined

    if submitted:   # cleaning the previous results
        clear_output()
        initialize_widgets()
    submitted = True
    output_df = pd.read_csv("output.csv")
    print("The results are presented in the table below. NaN values indicate that no faces have been detected on a picture.")
    display(output_df, clear)

submit.on_click(on_submit_clicked)
clear.on_click(on_clear_clicked)

In [5]:
text = HTML('<p>The results are saved in the <a href="output.csv" target="_blank">csv file</a> and the folder with photos is <a href="images" target="_blank">here</a> (in case you did not press on "Clear" yet, which deletes the stored data). </p>')
display(text)